This notebook shows how to download solar images (AIA cutouts) for a given jet in the Solar Jet Hunter catalogue.  
### To do before anything else
- In the directory `data`, create a subdirectory named `sjh_cutouts` if not done already. The data will be downloaded there.
- To download cutouts, you need to provide your email address. Please change it in the next line. You also need to register your email to the JSOC service [here](http://jsoc.stanford.edu/ajax/register_email.html) if you never did this before. JSOC is the provider of the data.

In [1]:
jsoc_email = "your.email.address.here@domain.com" ## Change this to your email address.

# Open catalogue and choose a jet

### Open catalogue
See tutorial for more details

In [8]:
import os
import numpy as np
from utils.Jet_class_light import json_import_list

In [3]:
json_file = 'exports/Jet_clusters_3.0_2.0.paperID.json'
Jet_catalogue=json_import_list(json_file)

The 883 JetCluster objects are imported from exports/Jet_clusters_3.0_2.0.paperID.json.


### Select a jet

In [4]:
jetid = 509
selected_jet = Jet_catalogue[jetid]

Print some of the jet's properties as a check

In [5]:
jet_start_time = selected_jet.obs_time
jet_end_time = selected_jet.obs_time + np.timedelta64(int(selected_jet.Duration*60), 's')
jet_sjh_identifier = selected_jet.ID
print(jet_start_time)
print(jet_end_time)
print(jet_sjh_identifier)

2015-09-22T21:56:06.000000
2015-09-22T22:27:42.000000
sjh_2015-09-22T21_1


### Extract the box information for this jet

In [12]:
from utils.Jet_box_class import make_cluster_box

In [13]:
jet_box = make_cluster_box(selected_jet)

### Path to the directory that will contain the data

In [6]:
data_path = 'data//sjh_cutouts//'+selected_jet.ID
print(data_path)

data//sjh_cutouts//sjh_2015-09-22T21_1


Check if this directory exists, and if not, create it

In [9]:
if not os.path.exists(data_path): os.mkdir(data_path)

# Download the data

### Prepare the download request

In [17]:
from astropy.coordinates import SkyCoord
import sunpy.coordinates  # Needed for helioprojective coordinate frame
from sunpy.net import Fido
from sunpy.net import attrs as a
import astropy.units as u

Calculate the coordinates of the field of view to be requested, from the jet box size and the jet base position

In [15]:
cutout_half_side = 60 if jet_box.height.value < 60 else jet_box.height.value + 20
x1 = jet_box.base[0].value - cutout_half_side
x2 = jet_box.base[0].value + cutout_half_side
y1 = jet_box.base[1].value - cutout_half_side
y2 = jet_box.base[1].value + cutout_half_side

### Query

In [18]:
bottom_left = SkyCoord(x1*u.arcsec, y1*u.arcsec, obstime=jet_start_time, observer="earth", frame="helioprojective")
top_right = SkyCoord(x2*u.arcsec, y2*u.arcsec, obstime=jet_start_time, observer="earth", frame="helioprojective")

In [19]:
cutout = a.jsoc.Cutout(bottom_left, top_right=top_right, tracking=False)

In [20]:
query = Fido.search(
    a.Time(jet_start_time, jet_end_time),
    a.Wavelength(304*u.angstrom),  ## Wavelength of the data shown in Solar Jet Hunter
    a.Sample(24*u.s),  ## this is to match the cadence of the data shown in Solar Jet Hunter
    a.jsoc.Series.aia_lev1_euv_12s,
    a.jsoc.Notify(jsoc_email),
    a.jsoc.Segment.image,
    cutout,
)
print(query)

Results from 1 Provider:

80 Results from the JSOCClient:
Source: http://jsoc.stanford.edu

       T_REC         TELESCOP INSTRUME WAVELNTH CAR_ROT
-------------------- -------- -------- -------- -------
2015-09-22T21:56:00Z  SDO/AIA    AIA_4      304    2168
2015-09-22T21:56:24Z  SDO/AIA    AIA_4      304    2168
2015-09-22T21:56:48Z  SDO/AIA    AIA_4      304    2168
2015-09-22T21:57:12Z  SDO/AIA    AIA_4      304    2168
2015-09-22T21:57:36Z  SDO/AIA    AIA_4      304    2168
2015-09-22T21:58:00Z  SDO/AIA    AIA_4      304    2168
2015-09-22T21:58:24Z  SDO/AIA    AIA_4      304    2168
2015-09-22T21:58:48Z  SDO/AIA    AIA_4      304    2168
2015-09-22T21:59:12Z  SDO/AIA    AIA_4      304    2168
2015-09-22T21:59:36Z  SDO/AIA    AIA_4      304    2168
                 ...      ...      ...      ...     ...
2015-09-22T22:23:36Z  SDO/AIA    AIA_4      304    2168
2015-09-22T22:24:00Z  SDO/AIA    AIA_4      304    2168
2015-09-22T22:24:24Z  SDO/AIA    AIA_4      304    2168
2015-09-22T2

### Start download

In [22]:
files = Fido.fetch(query, overwrite=False, path=data_path)